In [27]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

load_dotenv()

gtm_id = os.environ["GTM_ACCOUNT_ID"]

# Excel

In [28]:
xlsx_names = ['kyobo', 'library', 'scholar', 'story', 'tree']
service_names = {
    '교보문고_renewal': 'kyobo',
    '전자도서관': 'library',
    '스콜라': 'scholar',
    '스토리': 'story',
    '리딩트리': 'tree'
}

def get_xlsx():
    res = {}
    for name in service_names.values():
        res[name]=pd.ExcelFile(f'./xlsx/{name}.xlsx')

    return res

def get_sheet_name(excel):
    sheet_names = excel.sheet_names
    return sheet_names

def read_sheet(excel_file, sheet_name):
    return pd.read_excel(excel_file, sheet_name=sheet_name)

# GTM

In [29]:
# Get credentials
def get_credentials():
    SERVICE_ACCOUNT_FILE = './xlsx/gtm-api-409502-6f8362f190d7.json'

    credentials_container = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=["https://www.googleapis.com/auth/tagmanager.edit.containers"]
    )

    return credentials_container

credentials = get_credentials()

# Get service
def get_service():
    service = build('tagmanager', 'v2', credentials = credentials)

    return service

service = get_service()

# Get container
def get_container():
    parent = f"accounts/{gtm_id}"

    containers = service.accounts().containers().list(
        parent=parent,
    ).execute()

    return containers['container']

container_origin = get_container()
containers = {}
for i in container_origin:
    containers[i['name']] = i


# Set Container

In [30]:
# 교보문고_renewal, 전자도서관, 스콜라, 스토리, 리딩트리
service_name = '교보문고_renewal'
container = containers[service_name]

xlsxs = get_xlsx()
xlsx = xlsxs[service_names[service_name]]

# Get workspace
def get_workspaces():
    parent=container['path']

    workspace = service.accounts().containers().workspaces().list(
        parent=parent
    ).execute()

    return workspace['workspace']

workspace_origin = get_workspaces()

workspaces = {}
for i in workspace_origin:
    workspaces[i['name']] = i

workspace = workspaces['new']

# Get Elements

In [33]:
def get_trigger_list():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().triggers().list(
            parent=parent
        ).execute()

        print(f"Listed triggers: {response['trigger']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return e

def get_tag_list():
    parent=workspace['path']

    try:
        response = service.accounts().containers().workspaces().tags().list(
            parent=parent
        ).execute()

        print(f"Listed triggers: {response['tag']}")
        return response
    except Exception as e:
        print(f"An error occurred: {e}")
        return e


In [35]:
triggers = get_trigger_list()

trigger_obj = {}

for i in triggers['trigger']:
    trigger_obj[i['name'].lower()] = i['triggerId']

tags = get_tag_list()

tag_obj = {}

for i in tags['tag']:
    tag_obj[i['name'].lower()] = i['tagId']

Listed triggers: [{'path': 'accounts/6003640738/containers/90556056/workspaces/43/triggers/808', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'triggerId': '808', 'name': 'Event - sign_up', 'type': 'customEvent', 'customEventFilter': [{'type': 'equals', 'parameter': [{'type': 'template', 'key': 'arg0', 'value': '{{_event}}'}, {'type': 'template', 'key': 'arg1', 'value': 'sign_up'}]}], 'filter': [{'type': 'equals', 'parameter': [{'type': 'template', 'key': 'arg0', 'value': '{{DLV - platform}}'}, {'type': 'template', 'key': 'arg1', 'value': 'app'}, {'type': 'boolean', 'key': 'negate', 'value': 'true'}]}], 'fingerprint': '1704351198363', 'tagManagerUrl': 'https://tagmanager.google.com/#/container/accounts/6003640738/containers/90556056/workspaces/43/triggers/808?apiLink=trigger'}, {'path': 'accounts/6003640738/containers/90556056/workspaces/43/triggers/811', 'accountId': '6003640738', 'containerId': '90556056', 'workspaceId': '43', 'triggerId': '811', 'name': 

# Tags

In [37]:
tag_type_map = {
    'Google 태그': 'googtag',
    'Google 애널리틱스: GA4 이벤트': 'gaawe',
    '맞춤 HTML': 'html',
    'Firebase 이벤트 빌더': 'cvt_90556056_475',
    'Firebase 스크린 빌더': 'cvt_90556056_474',
}

def create_tag(request_body):
    parent=workspace['path']

    try:
        if request_body['name'].lower() in tag_obj:
            return
        response = service.accounts().containers().workspaces().tags().create(
            parent=parent,
            body=request_body
        ).execute()

        print(f"Created : {response['name']}", end="")

        time.sleep(4)

        return response
    except Exception as e:
        print('\n' + f"An error occurred: {e}")
        print(f"Above error occurred with: {request_body['name']}")
        return e


In [38]:
def create_tag_web(sheet):
    error = 0
    request_body = {}
    eventSettingsTable = {
        'type': 'list',
        'key': 'eventSettingsTable',
        'list': []
    }
    userProperties = {
        'type': 'list',
        'key': 'userProperties',
        'list': []
    }

    for index, row in sheet.iterrows():
        if type(row['태그명']) == str:
            if (index != 0):
                try:
                    request_body['parameter'].append(eventSettingsTable);
                    request_body['parameter'].append(userProperties);
                    create_tag(request_body)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['태그명'],
                "type": tag_type_map[row['태그 유형']],
                "parameter": [
                    {
                        'type': 'template', 
                        'key': 'measurementIdOverride', 
                        'value': row['측정 ID(태그 ID)']
                    }
                ],
                "firingTriggerId": [],
            }
            eventSettingsTable = {
                'type': 'list',
                'key': 'eventSettingsTable',
                'list': []
            }
            userProperties = {
                'type': 'list',
                'key': 'userProperties',
                'list': []
            }

            if type(row['이벤트 이름']) == str:
                request_body['parameter'].append({
                    'type': 'template', 
                    'key': 'eventName', 
                    'value': row['이벤트 이름']
                })

        if type(row['트리거']) == str:
            request_body['firingTriggerId'].append(trigger_obj[row['트리거'].lower()])

        if type(row['이벤트 매개변수(Event Settings Variable)']) == str:
            request_body['parameter'].append({
                'type': 'template', 
                'key': 'eventSettingsVariable', 
                'value': row['이벤트 매개변수(Event Settings Variable)']
            })

        if type(row['이벤트 매개변수']) == str:
            eventSettingsTable['list'].append({
                'type': 'map',
                'map': [
                    {
                        'type': 'template', 
                        'key': 'parameter', 
                        'value': row['이벤트 매개변수']
                    },
                    {
                        'type': 'template',
                        'key': 'parameterValue',
                        'value': sheet.iloc[index][7]
                    }
                ]
            })

        if type(row['사용자 속성']) == str:
            userProperties['list'].append({
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'name',
                        'value': row['사용자 속성']
                    },
                    {
                        'type': 'template', 
                        'key': 'value', 
                        'value': sheet.iloc[index][9]
                    }
                ]
            })

    try:
        create_tag(request_body)
        # print(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")



In [ ]:

{
    'name': 'GA4 - Event(App) - 화면 조회',
    'type': 'cvt_90556056_474',
    'parameter': [
        {
            'type': 'boolean',
            'key': 'eventParamsCheck',
            'value': 'true'
        },
        {
            'type': 'template',
            'key': 'eventParamsVar',
            'value': '{{(App)GA4 - Params - 공유}}'
        },
        {
            'type': 'boolean', 
            'key': 'activePageInfo', 
            'value': 'false'
        }
    ],
    'tagFiringOption': 'oncePerEvent',
}

{'name': 'GA4 - Event(App) - 화면 조회',
 'type': 'cvt_90556056_474',
 'parameter': [{'type': 'boolean', 'key': 'eventParamsCheck', 'value': 'true'},
  {'type': 'template',
   'key': 'eventParamsVar',
   'value': '{{(App)GA4 - Params - 공유}}'},
  {'type': 'boolean', 'key': 'activePageInfo', 'value': 'false'}],
 'tagFiringOption': 'oncePerEvent'}

In [ ]:
def create_tag_app(sheet):
    error = 0
    request_body = {}
    eventSettingsTable = {
        'type': 'list',
        'key': 'eventSettingsTable',
        'list': []
    }
    userProperties = {
        'type': 'list',
        'key': 'userProperties',
        'list': []
    }

    for index, row in sheet.iterrows():
        if type(row['태그명']) == str:
            if (index != 0):
                try:
                    request_body['parameter'].append(eventSettingsTable);
                    request_body['parameter'].append(userProperties);
                    create_tag(request_body)
                except Exception as e:
                    error += 1

            request_body = {
                "name": row['태그명'],
                "type": tag_type_map[row['태그 유형']],
                "parameter": [
                    {
                        'type': 'template', 
                        'key': 'measurementIdOverride', 
                        'value': row['측정 ID(태그 ID)']
                    }
                ],
                "firingTriggerId": [],
            }
            eventSettingsTable = {
                'type': 'list',
                'key': 'eventSettingsTable',
                'list': []
            }
            userProperties = {
                'type': 'list',
                'key': 'userProperties',
                'list': []
            }

            if type(row['이벤트 이름']) == str:
                request_body['parameter'].append({
                    'type': 'template', 
                    'key': 'eventName', 
                    'value': row['이벤트 이름']
                })

        if type(row['트리거']) == str:
            request_body['firingTriggerId'].append(trigger_obj[row['트리거'].lower()])

        if type(row['이벤트 매개변수(Event Settings Variable)']) == str:
            request_body['parameter'].append({
                'type': 'template', 
                'key': 'eventSettingsVariable', 
                'value': row['이벤트 매개변수(Event Settings Variable)']
            })

        if type(row['이벤트 매개변수']) == str:
            eventSettingsTable['list'].append({
                'type': 'map',
                'map': [
                    {
                        'type': 'template', 
                        'key': 'parameter', 
                        'value': row['이벤트 매개변수']
                    },
                    {
                        'type': 'template',
                        'key': 'parameterValue',
                        'value': sheet.iloc[index][7]
                    }
                ]
            })

        if type(row['사용자 속성']) == str:
            userProperties['list'].append({
                'type': 'map',
                'map': [
                    {
                        'type': 'template',
                        'key': 'name',
                        'value': row['사용자 속성']
                    },
                    {
                        'type': 'template', 
                        'key': 'value', 
                        'value': sheet.iloc[index][9]
                    }
                ]
            })

    try:
        create_tag(request_body)
        # print(request_body)
    except Exception as e:
        error += 1

    print(f"Total Error: {error}")



In [26]:
get_tag_list()

An error occurred: <HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/tags?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">


<HttpError 400 when requesting https://tagmanager.googleapis.com/tagmanager/v2/accounts/6003640738/containers/90556056/workspaces/43/tags?alt=json returned "Returned an error response for your request.". Details: "[{'message': 'Returned an error response for your request.', 'domain': 'global', 'reason': 'badRequest'}]">

In [40]:
sheet_names = get_sheet_name(xlsx)

for index, i in enumerate(sheet_names):
    print(index, i)

0 변수(당사쿠키)
1 변수(데이터 영역)
2 변수(URL)
3 변수(자동이벤트 변수)
4 변수(DOM 요소)
5 변수(맞춤JS)
6 변수(참고표)
7 변수(이벤트 매개변수)
8 트리거(맞춤 이벤트)
9 트리거(페이지뷰)
10 트리거(스크롤 깊이)
11 트리거(링크 클릭)(Web)
12 트리거(링크 클릭)(App)
13 트리거(모든 요소 클릭)(Web)
14 트리거(모든 요소 클릭)(App)
15 태그(맞춤HTML)
16  태그(Web)
17  태그(App 이벤트)


In [42]:
sheet_tag_web = read_sheet(xlsx, sheet_names[17])

create_tag_web(sheet_tag_web)

KeyError: '측정 ID(태그 ID)'